In [13]:
import pandas as pd
import numpy as np
df_final_demo = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_demo.txt")
df_final_experiment_clients = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_experiment_clients.txt")
df_final_web_data_pt_1 = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_web_data_pt_1.txt")
df_final_web_data_pt_2 = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_web_data_pt_2.txt")


----------------------

                    -- Concating and merging 3 files to df_final_web_data

----------------------

In [2]:
# Concating df_final_web p1 and p2 and merging them with df_final_experiment_clients
df_final_web_data = pd.concat([df_final_web_data_pt_1, df_final_web_data_pt_2], ignore_index=True)

# Merging three tables 
df_web_experiment = pd.merge(
    df_final_web_data,
    df_final_experiment_clients,
    on="client_id",
    how="inner"
)

----------------------

                    -- Cleaning and sorting df_web_experiment

----------------------

In [3]:
# cleaning and droping duplicated and null values in Variation 
df_web_experiment = df_web_experiment.drop_duplicates()
# df_web_experiment = df_web_experiment.dropna(subset=['Variation'])

# replacing steps by numbers 
df_web_experiment["process_step"] = (df_web_experiment["process_step"].replace({
    "start": 1,
    "step_1": 2,
    "step_2": 3,
    "step_3": 4,
    "confirm": 5
}).astype(int))

# fixing date_time 
df_web_experiment['date_time'] = pd.to_datetime(df_web_experiment['date_time'], errors='coerce')

# sorting by client, visit and date_time
df_web_experiment = df_web_experiment.sort_values(
    by=['client_id', 'visit_id', 'date_time'],
    ascending=[True, True, True]
)

/var/folders/lt/yxd_bdjs4fn9bd90z539wlsm0000gn/T/ipykernel_12054/2385145628.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_web_experiment["process_step"] = (df_web_experiment["process_step"].replace({


----------------------

                    -- Adding new columns by group by to df_web_experiment

----------------------

In [4]:
# adding time_from_prev_step per second
df_web_experiment['time_from_prev_step'] = (
    df_web_experiment
    .groupby('visit_id')['date_time']
    .diff()                           
    .dt.total_seconds() 
    .astype('Int64')  
)

# adding time_from_start
df_web_experiment['time_from_start'] = (
    df_web_experiment
    .groupby('visit_id')['date_time']
    .transform(lambda x: x - x.min()) 
    .dt.total_seconds()
    .astype('Int64')  
)

# adding num_stpes
df_web_experiment['num_steps'] = (
    df_web_experiment
    .groupby('visit_id')['process_step']
    .transform('count')
)

# adding num of visit per client 
df_web_experiment['num_visits'] = (
    df_web_experiment
    .groupby('client_id')['visit_id']
    .transform('nunique')
)

In [5]:
df_web_experiment_visits  = df_web_experiment.drop_duplicates(subset='visit_id', keep='last')


----------------------

                    -- Merging all of them together

----------------------

In [6]:
# Merging all together 
df_raw = pd.merge(
    df_web_experiment,
    df_final_demo,
    on="client_id",
    how="left"
)

----------------------

                    -- Cleaning df_raw

----------------

In [7]:
# making columns int and rounding "bal"
df_raw = df_raw.copy()

# columns i want to change to int
int_cols = ['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age',
            'num_accts', 'calls_6_mnth', 'logons_6_mnth']

# to round it
float_col = 'bal'

# trasnforming the columns 
for col in int_cols:
    df_raw[col] = (
        pd.to_numeric(df_raw[col], errors='coerce')                                         
        .round()                                           
        .astype('Int64')                                  
    )

# rounding float "bal"
df_raw[float_col] = (
    pd.to_numeric(df_raw[float_col], errors='coerce')
    .round(2)
)

# fixing date_time
df_raw['date_time'] = pd.to_datetime(df_raw['date_time'], errors='coerce')

# sorting by client, visit and date_time
df_raw = df_raw.sort_values(
    by=['client_id', 'visit_id', 'date_time'],
    ascending=[True, True, True]
)


----------------------

                        -- Making df_raw_visits which has the unique and last visit_id to see the last step they took 

----------------------

In [8]:
# df with the unique visit id with Variation
df_raw_visits = df_raw.drop_duplicates(subset='visit_id', keep='last')


In [15]:
# adding extra_step to see how many steps it took more that 5 to complete 
df_raw_visits = df_raw_visits.copy()
df_raw_visits['extra_step'] = np.where(
    df_raw_visits['process_step'] == 5,
    df_raw_visits['num_steps'] - 5,
    0
)

In [16]:
df_raw_visits

,client_id,visitor_id,visit_id,process_step,date_time,Variation,time_from_prev_step,time_from_start,num_steps,num_visits,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,extra_step
4,169,201385055_71273495308,749567106_99161211863_557568,5,2017-04-12 20:23:09,NaN,64,213,5,1,21,262,48,M,2,501570.72,4,4,0
9,555,402506806_56087378777,637149525_38041617439_716659,5,2017-04-15 13:00:34,Test,20,158,5,1,3,46,30,U,2,25454.66,2,6,0
14,647,66758770_53988066587,40369564_40101682850_311847,5,2017-04-12 15:47:45,Test,163,377,5,1,12,151,58,M,2,30525.80,0,4,0
23,722,919259913_64837298108,984487154_55831795985_521110,5,2017-04-19 15:00:09,NaN,162,599,9,1,11,143,60,F,2,22466.17,1,1,4
27,934,810392784_45004760546,7076463_57954418406_971348,1,2017-04-18 02:38:52,Test,28,142,4,1,9,109,51,F,2,32522.88,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443881,9999729,843385170_36953471821,493310979_9209676464_421146,1,2017-04-20 14:28:57,Test,81,450,4,3,10,124,31,F,3,107059.74,6,9,0
443886,9999729,834634258_21862004160,870243567_56915814033_814203,5,2017-05-08 16:09:40,Test,21,75,5,3,10,124,31,F,3,107059.74,6,9,0
443888,9999729,604429154_69247391147,99583652_41711450505_426179,2,2017-04-05 13:41:04,Test,15,15,2,3,10,124,31,F,3,107059.74,6,9,0
443890,9999832,145538019_54444341400,472154369_16714624241_585315,2,2017-05-16 16:46:11,Test,8,8,2,1,23,281,49,F,2,431887.61,1,4,0
